In [1]:
import pandas as pd
import numpy as np
import re

동별 매력도 feature 추출

In [ ]:
temp  = pd.read_csv('/home/jovyan/WIDE_HOME/CompetitionData/선도소프트/population_move(2020).csv')
temp = temp[temp['TRNSFRN_CTPRVN']=='서울특별시']
pivot = pd.pivot_table(temp,index ="TRNSFRN_EMD" , columns ="TRNSFRN_REASON", values = "TRNSFRN_CTPRVN", aggfunc = 'count' )
pivot.drop(['가족', '건강', '교육', '기타'], axis = 1, inplace = True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
Scaler.fit(pivot)
Scaled = Scaler.transform(pivot)
pivot2 = pd.DataFrame(Scaled, index = pivot2.index, columns = pivot2.columns )
pivot2.to_csv('/home/jovyan/WIDE_HOME/동별매력지수.csv')

동별 사업체 밀도 feature 추출

In [ ]:
url = '/home/jovyan/WIDE_HOME/동별 사업체 및 종사자 밀도.txt'
dataset = pd.read_csv(url, sep = '\t')
dataset.drop(['기간', '자치구','사업체수', '종사자수', '평균종사자','종사자밀도(A)',
       '인구밀도(B)', '종사자밀도비(A/B*100)'], axis = 1, inplace = True)
dataset.to_csv('/home/jovyan/WIDE_HOME/동별사업체밀도.csv')

동별 사업체 창업률 feature 추출

In [ ]:
url = '/home/jovyan/WIDE_HOME/동별 사업체 창업률.txt'
dataset = pd.read_csv(url, sep = '\t')
dataset.drop(['자치구','기간', '사업체수', '종사자수','평균종사자'], axis = 1, inplace = True)
dataset.drop([0,1], axis = 0, inplace = True)
dataset.index = range(448)
dataset.to_csv('/home/jovyan/WIDE_HOME/동별창업매력도.csv')

지리적 거리도 추출

In [ ]:
data  = pd.read_csv('/home/jovyan/WIDE_HOME/유수빈/빌딩+평당금액.csv')
long_distance = pd.DataFrame(index = range(1985), columns = range(1985))
lati_distance = pd.DataFrame(index = range(1985), columns = range(1985))

long = data['경도']
lati = data['위도']

for i in range(1985):
    for j in range(1985):
        long_distance[i][j] = abs(long[i] - long[j])
        
for i in range(1985):
    for j in range(1985):
        lati_distance[i][j] = abs(lati[i] - lati[j])
        
geographic_distance = (long_distance*100)*(lati_distance*100) #거리곱은 매우 작아지기 때문에, 미리 스케일링을 위해 100씩 곱해줌
geographic_distance.to_csv('./지리적거리도.csv')

Features load 및 Dataset 구성

In [2]:
data  = pd.read_csv('/home/jovyan/WIDE_HOME/유수빈/빌딩+평당금액.csv') #평당금액까지 합친 dataset을 load합니다

In [3]:
Attractor = pd.read_csv('/home/jovyan/WIDE_HOME/동별매력지수.csv' ) #서울 열린 데이터광장 동별 전입지 이동사유별 인구이동 통계

In [4]:
Attractor.columns = ['동이름', '교통', '주택', '직업']

In [5]:
Attractor['동이름'] = Attractor['동이름'].str.replace(r'[0-9]+', '')

In [6]:
Attractor.index = Attractor['동이름']
Attractor.drop(['동이름'], axis = 1, inplace = True)

In [7]:
교통_dict = Attractor['교통'].to_dict()
주택_dict = Attractor['주택'].to_dict()
직업_dict = Attractor['직업'].to_dict()

In [8]:
data['교통매력도'] = data['법정동명'].map(교통_dict)
data['주택매력도'] = data['법정동명'].map(주택_dict)
data['직업매력도'] = data['법정동명'].map(직업_dict)

In [9]:
Potential_customers =  pd.read_excel('/home/jovyan/WIDE_HOME/동별생활인구데이터.xlsx' ) #서울시 생활인구 통계서비스 데이터
Potential_customers['장년층'] = Potential_customers[['연령대_30_생활인구_수','연령대_40_생활인구_수', '연령대_50_생활인구_수']].transpose().mean()

In [10]:
Potential_customers.drop(['남성_생활인구_수', '여성_생활인구_수','연령대_30_생활인구_수','연령대_40_생활인구_수', '연령대_50_생활인구_수'], axis = 1, inplace = True)

In [11]:
Potential_customers.columns = ['동이름', '총생활인구', '청소년층', '청년층', '장년층']

In [12]:
Potential_customers.index = Potential_customers['동이름']
Potential_customers.drop(['동이름'], axis = 1, inplace = True)

In [13]:
총생활_dict = Potential_customers['총생활인구'].to_dict()
청소년_dict = Potential_customers['청소년층'].to_dict()
청년_dict = Potential_customers['청년층'].to_dict()
장년_dict = Potential_customers['장년층'].to_dict()

In [14]:
data['총생활인구'] = data['법정동명'].map(총생활_dict)
data['청소년생활인구'] = data['법정동명'].map(청소년_dict)
data['청년생활인구'] = data['법정동명'].map(청년_dict)
data['장년생활인구'] = data['법정동명'].map(장년_dict)

In [15]:
Density = pd.read_csv('/home/jovyan/WIDE_HOME/동별사업체밀도.csv' )
Density.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [16]:
Density['동'] = Density['동'].str.replace(r'[0-9]+', '')
Density['사업체밀도'] = Density['사업체밀도'].str.replace(r',', '')
Density['사업체밀도'] = Density['사업체밀도'].astype('int')

In [17]:
Density.drop([0,1], axis = 0, inplace = True)
Density = Density.groupby('동').mean()
사업체밀도dict = Density['사업체밀도'].to_dict()

In [18]:
data['사업체밀도'] = data['법정동명'].map(사업체밀도dict)

In [19]:
#사업체창업률  = pd.read_csv('/home/jovyan/WIDE_HOME/동별창업매력도.csv')

In [20]:
#start_up_rate = pd.read_csv('/home/jovyan/WIDE_HOME/동별창업매력도.csv') #서울시 열린데이터광장 동별 창업률 데이터
#start_up_rate['동'] = start_up_rate['동'].str.replace(r'[0-9]+', '')

In [21]:
#창업률dict = start_up_rate['사업체 창업률'].to_dict()
#data['창업률'] = data['법정동명'].map(창업률dict)

Null 값 제거

In [22]:
Null_square = data[['경도', '위도','사업체 창업률','교통매력도', '주택매력도', '직업매력도', '총생활인구', '청소년생활인구',
       '청년생활인구', '장년생활인구', '사업체밀도']]

In [23]:
Geographic_distance_matrix = pd.read_csv('/home/jovyan/WIDE_HOME/이한울/지리적거리도.csv') #1985개 건물의 거리차이를 나타낸 1985 x 1985 matrix

In [24]:
Geographic_distance_matrix.drop(['Unnamed: 0'], axis = 1, inplace = True)
Geographic_distance_matrix = Geographic_distance_matrix.astype('float')

In [25]:
Geographic_distance_matrix.columns = Geographic_distance_matrix.columns.astype('int')
Geographic_distance_matrix.index = Geographic_distance_matrix.index.astype('int')

In [26]:
Nearby = pd.DataFrame(index = range(1985), columns = ['near'])

In [ ]:
for j in range(1,15): #경,위도 거리를 바탕으로 Null 값 처리
    Near_plot = Geographic_distance_matrix < j #점점 허용가능한 거리를 확장
    for i in range(1985):
        Nearby['near'][i] = Near_plot.index[Near_plot[i]].to_list() #허용가능한 거리 내의 건물들 index 확인
    for column in Null_square.columns:
        for index in Null_square.index:
            if np.isnan(Null_square[column][index]):
                for each in Nearby['near'][index]:
                    empty_list = []
                    empty_list.append(Null_square[column][each]) 
                Null_square[column][index] = np.array(empty_list).mean() #허용가능한 거리 내의 건물들의 feature값들을 평균


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
Null_square['법정동명'] = data['법정동명']
Null_square['동코드'] = data['법정동 동 코드 '].astype('int')
Check_point = ['사업체 창업률', '교통매력도', '주택매력도', '직업매력도', '총생활인구', '청소년생활인구',
       '청년생활인구', '장년생활인구', '사업체밀도']

In [ ]:
for column in Check_point: #같은 법정동 내에 위,경도 지리도를 통해 채워진 Null 값이 있을 경우, 해당 값들을 평균내어 같은 법정동 내 Null 값 제거
    for index in range(1985):
        if np.isnan(Null_square[column][index]) :
            법정동 = Null_square['법정동명'][index]
            To_check = Null_square[Null_square['법정동명']==법정동]
            Null_square[column][index] = To_check[column].mean()
            

In [ ]:
for column in Check_point: #법정동 코드 상 인접 구역의 값들로 Null 값 제거
    for index in range(1985):
        if np.isnan(Null_square[column][index]) :
            Code = Null_square['동코드'][index]
            Codes = np.array([Code, Code+100, Code-100, Code+200, Code-200, Code+300, Code-300])
            for dong in Codes:
                Matrix = Null_square[Null_square['동코드']== dong]
                if np.isnan(Matrix[column].mean()):
                    pass
                elif np.isnan(Null_square[column][index]):
                    Null_square[column][index] = Matrix[column].mean()
                else:
                    pass
      

In [ ]:
data['사업체 창업률'] = Null_square['사업체 창업률']
data['교통매력도'] = Null_square['교통매력도']
data['주택매력도'] = Null_square['주택매력도']
data['직업매력도'] = Null_square['직업매력도']
data['총생활인구'] = Null_square['총생활인구']
data['청소년생활인구'] = Null_square['청소년생활인구']
data['청년생활인구'] = Null_square['청년생활인구']
data['장년생활인구'] = Null_square['장년생활인구']
data['사업체밀도'] = Null_square['사업체밀도']

In [ ]:
data.to_csv('/home/jovyan/WIDE_HOME/Null제거_dataset2.csv')